In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14671320
paper_name = 'kushner_ahlquist_2003' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [11]:
original_data = pd.read_excel('raw_data/6857Table3.xlsx', sheet_name='Sheet1', skiprows=10)

In [12]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4792 x 9


In [13]:
original_data.head()

,ORF,name,% wt Rluc,activity 1,notes 1,notes 1.1,activity 2,notes 2,notes 2.1
0,YAL002W,VPS8,163.919012,166.080946,,,161.757077,,
1,YAL004W,NaN,157.897736,198.830915,,,116.964557,,
2,YAL005C,SSA1,195.307831,231.299046,,,159.316616,,
3,YAL007C,ERP2,69.358685,67.896887,,g,70.820482,,g
4,YAL008W,FUN14,109.664038,96.854861,,,122.473215,,


In [14]:
original_data['orf'] = original_data['ORF'].astype(str)

In [15]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [16]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [17]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, name, % wt Rluc, activity 1, notes 1, notes 1.1, activity 2, notes 2, notes 2.1, orf]
Index: []


In [18]:
original_data['data'] = pd.to_numeric(original_data['% wt Rluc'], errors='coerce')

In [19]:
original_data.set_index('orf', inplace=True)

In [20]:
original_data = original_data[['data']].copy()

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(4679, 1)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [481]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,481
data_type,value
orf,
YAL002W,163.919012
YAL004W,157.897736
YAL005C,195.307831
YAL007C,69.358685
YAL008W,109.664038


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 27


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,481
,data_type,value
gene_id,orf,
2,YAL002W,163.919012
1863,YAL004W,157.897736
4,YAL005C,195.307831
5,YAL007C,69.358685
6,YAL008W,109.664038


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id              481          
data_type             value    valuez
gene_id orf                          
2       YAL002W  163.919012  1.318740
1863    YAL004W  157.897736  1.198743
4       YAL005C  195.307831  1.944278
5       YAL007C   69.358685 -0.565725
6       YAL008W  109.664038  0.237508

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 14671320...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.81s/it]

Updating the data_modified_on field...
